# **2. Texto Preditivo, Parte I: N-Gramas**

<div>
<img src="../assets/autocomplete.png" width="500" style=" display: block; margin-left: auto; margin-right: auto;"/>
</div>

Nessa seção, construiremos **texto preditivo**, um sistema que sugere a palavra seguinte enquanto um usuário digita. Texto preditivo é usado em teclados de celular, aplicativos de busca, composição por e-mail com força IA e muito mais.

O objetivo principal de um sistema de texto preditivo é *dada uma sequência de palavras, prever a próxima palavra mais provável*.

#### Como fazemos isso?
Em Processamento de Linguagem Natural (PLN), isso pode ser resolvido com um **modelo de idioma**. Um modelo de idioma aprende a distribuição de palavras no texto. Vamos construir um modelo de linguagem simples que aprende frases comuns de duas ou três palavras.
***

## **Data Preparation**
### Carregar o corpus
Para esta aplicação, utilizamos texto em inglês do corpus COCA. 

<div class="alert alert-block alert-warning">
    Como antes, sinta-se à vontade para usar sua própria língua.
</div>

In [ ]:
# Load our data
import util

# REPLACE WITH YOUR CORPUS DIRECTORY
corpus = util.load_raw_text(corpus_directory="../../corpora/eng")
corpus[:1000]

### Pré-processamento e Tokenização
Como no corretor automático, o próximo passo é de tokenizar o texto em palavras individuais. A única diferença aqui é que mantemos a pontuação.

#### **Exercício 1**
Use o regex fornecido para tokenizar o texto e retornar o resultado.

detalhes
  <summary>Mostrar resposta</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">return re.findall(word_or_punctuation_regex, text)</code></pre>
</details>

In [ ]:
import re

word_or_punctuation_regex = r"[\w|\']+|[\.|\,|\?|\!]"

def preprocess(text):
    text = util.strip_accents(text)
    text = text.lower()

    # TODO: Use the provided regex to tokenize the text, and return the result

tokens_filtered = preprocess(corpus)

print(len(tokens_filtered), "total tokens")
print(tokens_filtered[:200])

## **N-Gram Modelamento**
Agora, vamos construir nosso primeiro modelo. Para isso, usaremos *n-gramas*.

Um n-grama é simplesmente uma sequência de *n* palavras que ocorre em nosso texto. Por exemplo, considere a frase:

Como é seu nome?

Se tivéssemos a lista de todos os *bigramas* (2 palavras cada) na frase, teríamos:

- *Como é*
- *é seu*
- *seu nome*
- *nome ?*

Se tivéssemos a lista de todos os *bigramas* (3 palavras cada) na frase, teríamos:

- *Como é seu*
- *é seu nome*
- *seu nome* ?*

E assim por diante.

### Usando N-gramas como modelo de idioma

Podemos facilmente usar a ideia de n-gramas para construir um modelo de idioma simples. Por exemplo, se estamos tentando prever a próxima palavra, usando trigramas, dado a entrada:

> Como é seu* ...

Podemos olhar todos os trigramas que começam com *é seu*, e escolher o mais comum. Vamos usar nosso texto tokenizado para obter uma lista de todos os n-gramas que aparecem no texto. 

### Preenchimento de frases
Neste momento, teríamos n-gramas que passariam os limites das frases, como "manchetes de você". Isso não é muito útil, então uma técnica comum é **preencher** cada frase com os tokens representando o início da frase.

In [ ]:
def pad(text: list, num_padding: int):
    
    padded_text = []
    
    # Add initial padding to the first sentence
    for _ in range(num_padding):
        padded_text.append("<s>")
    
    for word in text:
        padded_text.append(word)

        # Every time we see an end punctuation mark, add <s> tokens after it
        # REPLACE IF YOUR LANGUAGE USES DIFFERENT END PUNCTUATION
        if word in [".", "?", "!"]:
            for _ in range(num_padding):
                padded_text.append("<s>")
        
        
    return padded_text

print(pad(tokens_filtered, 2)[:30])

### Criar uma lista de n-gramas
O código a seguir usa a biblioteca **NLTK** para criar uma lista de trigramas.

#### **Exercício 2**
Em vez disso, o que mudaríamos para utilizar o bigrama?

<details>
  <summary>Mostrar resposta</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">padded_tokens = pad(tokens_filtered, 1)
trigrams = list(ngrams(sequence=padded_tokens, n=2))</code></pre>
</details>

In [ ]:
from nltk.util import ngrams

# Now, we can actually create the list of n-grams using the NLTK library
padded_tokens = pad(tokens_filtered, 2)
trigrams = list(ngrams(sequence=padded_tokens, n=3))
trigrams[:30]

Agora que temos uma lista de trigramas, podemos contar a frequência de cada trigrama diferente.

#### **Exercício 3**
Usando a lista de trigramas, preencha o dicionário `todos_trigrams`, tal que cada chave é um trigrama único e o valor é quantas vezes esse trigrama ocorre no texto.

<details>
  <summary>Mostrar resposta</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">for gram in trigrams:
    if gram in all_trigrams:
        all_trigrams[gram] += 1
    else:
        all_trigrams[gram] = 1</code></pre>
</details>

In [ ]:
# A dict of all trigrams and their frequency
all_trigrams = dict()

# TODO: Add each unique trigram to the dictionary and set the value to how many times that trigram occurs in the text
        
len(all_trigrams)

In [ ]:
# Let's see what the twenty most common trigrams are
sorted(all_trigrams.items(), key=lambda x: x[1], reverse=True)[:30]

## Fazer previsões usando n-gramas

Agora que temos uma contagem de todos os trigramas, podemos fazer previsões procurando o trigrama mais comum que corresponda ao nosso input. 

In [ ]:
def predict_trigram_model(text, number_results = 3):
    input_tokens = pad(preprocess(text), 2)
    
    # Find the last 2 tokens in the input
    last_two_tokens = input_tokens[-2:]
    
    # Search our list of all trigrams to find matching trigrams
    matching_trigrams = []
    for item in all_trigrams.items():
        gram = item[0]
        
        # Check if the first and second item in the trigram are a match
        if gram[0] == last_two_tokens[0] and gram[1] == last_two_tokens[1]:
            matching_trigrams.append(item)
    
    # Now, sort the matching trigrams by popularity and return the first `number_results` results
    sorted_matching_trigrams = sorted(matching_trigrams, key=lambda x: x[1], reverse=True)
    top_matching_trigrams = sorted_matching_trigrams[:number_results]
    
    # Last, let's just get the predicted word (the last word of the trigram)
    predictions = [trigram[0][2] for trigram in top_matching_trigrams]
    return predictions

predict_trigram_model("how is")

## Tornar o texto preditivo um app autônomo

#### **Exercício 4**
Crie uma UI usando a função `predict_trigram_model`. Consulte o corretor automático para obter ajuda.

<details>
  <summary>Mostrar resposta</summary>
      <pre style="background-color: honeydew; padding: 10px; border-radius: 5px;"><code style="background: none;">autocompleter = gr.Interface(fn=predict_trigram_model, inputs="text", outputs="text", live=True)
</code></pre>
</details>

In [ ]:
import gradio as gr
gr.close_all()

autocompleter = None

# TODO: Create a UI using Gradio for predictive text

autocompleter.launch(share=True)

# **Melhorar N-Gramas com Backoff**
Nosso aplicativo tem um problema importante. Se a frase que inserimos termine com duas palavras que não correspondem a *qualquer* trigrama, então o nosso modelo não tem previsões a fazer.

Uma solução comum para isso é usar um processo chamado **backoff**. Com Backoff, se o nosso modelo de trigrama não encontrar nenhum resultado, então tentaremos encontrar uma correspondência de *bigramas*. Se isso não der certo, vamos para *unigramas* (o que significa que usamos apenas as palavras mais frequentes). Isso significa que sempre obtemos um resultado, mesmo que não seja tão bom.

Abaixo está um modelo que usa retrocessos e um n-gram de tamanho arbitrário para fazer previsões, agrupando tudo até agora.

In [ ]:
n_gram_models = dict()

def create_ngram_model(n = 3):
    padded_tokens = pad(tokens_filtered, n - 1)
    grams = list(ngrams(sequence=padded_tokens, n=n))
    
    all_ngrams = dict()
    for gram in grams:
        if gram in all_ngrams:
            all_ngrams[gram] += 1
        else:
            all_ngrams[gram] = 1
    return all_ngrams


def predict_ngram_model(text, n = 3, number_results = 3):
    input_tokens = pad(preprocess(text), n - 1)
    
    while n > 0:
        # Find the last n - 1 tokens in the input
        last_tokens = tuple(input_tokens[-(n-1):])
    
        if not n in n_gram_models:
            n_gram_models[n] = create_ngram_model(n)
        
        matching_ngrams = []
        
        for item in n_gram_models[n].items():
            gram = item[0]
            if gram[:-1] == last_tokens:
                matching_ngrams.append(item)
    
        # Now, sort the matching n-grams by popularity and return the first `number_results` results
        sorted_matching_ngrams = sorted(matching_ngrams, key=lambda x: x[1], reverse=True)
        top_matching_ngrams = sorted_matching_ngrams[:number_results]
        
        # BACKOFF: If there are no results, drop n and try again
        if len(top_matching_ngrams) == 0:
            print("backing off to:", n-1)
            n = n - 1
            continue
    
        # Last, let's just get the predicted word (the last word of the trigram)
        predictions = [gram[0][-1] for gram in top_matching_ngrams]
        return predictions
    return []

predict_ngram_model("why is", 4)

## **Resumo**
Neste tutorial, construímos uma ferramenta preditiva para uma língua com poucos recursos. Isso incluiu:
- Criar uma lista de n-gramas do corpus
- Usar n-gramas para prever a próxima palavra em uma sequência
- Usar o backoff para utilizar n-gramas de tamanhos diferentes conforme necessário

### **Desafios**
1. Use a função de previsão repetidamente para prever várias sequenciais de palavras, a partir de alguma sequência. Por exemplo, a sequência "O que" deve ser seguida de "ele faz" ou "ela faz".
1. Melhore o aplicativo de texto preditivo com botões que permitem adicionar uma sugestão ao texto que você está digitando.
1. É um pouco chato mostrar sempre as previsões mais comuns. Adicione um elemento de aleatoriedade usando a biblioteca `random` do Python, para que a função de previsão não mostre sempre os mesmos resultados.